In [ ]:
# Internet Repositories used in our research

'This repository was used from us to have a better understanding on how to incorporate Markovify and use the rhyme system'
'https://www.kaggle.com/code/alcoach/arabic-poem-generator/notebook'

'This repository was used from us to figure out the struxture and how to build and combine an LSTM model and Marovify'
'https://github.com/helloMinji/AI-RAPSTAR/blob/master/model.py'

'This repository was used from us to gain a better understanding on how to configure the LSTM model'
'https://colab.research.google.com/drive/1wlZXZBvOo93pAmTtEUeTlPsgAP4D1bLA#scrollTo=I5EngGk8YuJv'

'This code found on StackOverflow helped us understand and code the count of syllables'
'https://stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word'

'https://stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word'

In [ ]:
# Taking care of dependencies needed throughout the code
!pip install markovify
!pip install pronouncing
# Imports
import markovify
import re
import pronouncing
import random
import numpy as np
import os
from keras.models import Sequential
from keras.layers import LSTM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 13.4 MB/s 
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18628 sha256=ad91a9a80fd6eb29af564f75264b2b229d04dd47810c00e4c47064f31f8923b5
  Stored in directory: /root/.cache/pip/wheels/36/c5/82/11125c5a7dadec27ef49ac2b3a12d3b1f79ff7333c92a9b67b
Successfully built markovify
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 939 kB 8.7 MB/s 
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6252 sha256=162e38e3f4ea51790a163eb7c595404526904944e9ac5734db63f537df55431e
  Stored in directory: /root/.cache/pip/wheels/09/e8/c0/3606d42fdbf5f3871564eb6a353591a8f5deeed013fdb73921
Successfully built pronouncing


In [ ]:
# Load dataset in .txt format and UTF8 encoding /content/sample_data/Bars.txt
with open('Bars.txt', encoding="utf8") as f:
    noted_rap = f.read()
# Split the dataset into lines
ed_rap = [x.split("\r")[0] for x in noted_rap.split("\n")]
# Remove spaces
while "" in ed_rap:
  ed_rap.remove("")
while " " in ed_rap:
  ed_rap.remove(" ")

In [ ]:
# Test: print first 10 rap lines to be sure dataset is correctly loaded
print(f'{ed_rap[:10]}')

['\ufefftext', 'Heartbreak drowned sorrows in a large steak', 'Why you always all on my back?', 'Why you gotta do me like that?', "Why you gotta act like a bitch when I'm with you?", "Baby girl I'm blue", 'Because you treat me like shit', 'I paid for the bed and never even slept in it', 'I paid for that crib I never stepped foot in', 'And now somebody else is eating all the pudding']


In [ ]:
def make_model(depth):
    '''Function that takes as input the depth of the network.
     Returns the model'''
    model = Sequential()
    # Add a LSTM layer with 4 units with input_shape of (2,2)
    model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))
    # Add as many number of layers as depth. Each layer has 8 nodes
    for i in range(depth):
        model.add(LSTM(8, return_sequences=True))
    # Adds a LSTM layer with 2 nodes (output)
    model.add(LSTM(2, return_sequences=True))
    # Set up RMSprop as the optimizer and MSE as the loss function
    model.compile(optimizer='rmsprop',
                  loss='mse')
    return model

In [ ]:
def syllables(line):
    '''Check that length of a bar is not longer than maxsyllables.
       Generate bars until the number of syllables is less than maxsyllables.'''
    cntr = 0
    for word in line.split(" "):
        vowels = 'aeiouy'
        word = word.lower().strip(".:;?!")
        if not word:
            continue
        if word[0] in vowels:
            cntr += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                cntr += 1
        if word.endswith('e'):
            cntr -= 1
        if word.endswith('le'):
            cntr += 1
        if cntr == 0:
            cntr += 1
    return cntr / maxsyllables

In [ ]:
def rhymeindex(lyrics):
    '''Build a rhyme list. Write the list on a file.'''
    # Check if the list is already made
    if str('list') + ".rhymes" in os.listdir("."):
        print('Loading rhymes from ' + str('list') + ".rhymes")
        return open(str('list') + ".rhymes", "r",encoding='utf-8').read().split("\n")
    # If not, build list of rhymes
    else:
        nrlist = []
        for i in lyrics:
            word = re.sub(r"\W+", '', i.split(" ")[-1]).lower()
            rhymeslist = pronouncing.rhymes(word)
            rhymeslistends = []      
            for i in rhymeslist:
                rhymeslistends.append(i[-2:])
            try:
                rhymescheme = max(set(rhymeslistends), key=rhymeslistends.cntr)
            except Exception:
                rhymescheme = word[-2:]
            nrlist.append(rhymescheme)
        # Use set() to make the list unique
        nrlist = list(set(nrlist))
        reverselist = [x[::-1] for x in nrlist]
        # Sort reverselist
        reverselist = sorted(reverselist)
        rhymelist = [x[::-1] for x in reverselist]
        f = open(str('list') + ".rhymes", "w", encoding='utf-8')
        f.write("\n".join(rhymelist))
        f.close()
        return rhymelist

In [ ]:
def rhyme(line, rhyme_list):
    '''Find most common rhyme ending using max() function.
       If found, convert to float'''
    word = re.sub(r"\W+", '', line.split(" ")[-1]).lower()
    rhymeslist = pronouncing.rhymes(word)
    rhymeslist = [x.encode('UTF8') for x in rhymeslist]
    rhymeslistends = []
    for i in rhymeslist:
        rhymeslistends.append(i[-2:])
    try:
        # Use set() to choose unique rhymes
        rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
    except Exception:
        rhymescheme = word[-2:]
    try:
        float_rhyme = rhyme_list.index(rhymescheme)
        float_rhyme = float_rhyme / float(len(rhyme_list))
        return float_rhyme
    except Exception:
        return 0

In [ ]:
def get_last_word(bar):
    '''Get the last word of the bar'''
    for last_word in bar.split(" "):
        last_word = bar.split(" ")[-1]
        if len(last_word) == 0:
          last_word = bar.split(" ")[-2]
        # Check if the last word is punctuation.
        # If this is the case, get the word before it
        if not last_word[-1] in "/!.?,":
            continue
        if last_word[-1] in "/!.?,":
            last_word = last_word[:-1]
    return last_word

In [ ]:
def generate_lyrics(lyrics_file):
    '''Using Markov model (markovify), generate lyrics'''
    bars = []
    last_words = []
    linelength = len(lyrics_file)
    cntr = 0
    markov_model = markovify.NewlineText(ed_rap)
    #markov_model = markov((". ").join(lyrics_file) + ".")

    # While the length of the bars is less than linelength/9
    # and the counter is less than double the linelength, return a bar
    # (We use the while loop to to prevent overlapping of the newly generated text
    # and the original one).
    while len(bars) < linelength / 9 and cntr < linelength * 2:
        bar = markov_model.make_sentence()

        # Check that syllables is under the max number of syllables
        if type(bar) != type(None) and syllables(bar) < 1 and len(bar.split(" ")[-1]) != 0:
            last_word = get_last_word(bar)
            # Check if the bar is unique and that last_word is not overused
            # (In this setting, we set maximum number of times as 3)
            if bar not in bars and last_words.count(last_word) < 3:
                bars.append(bar)
                last_words.append(last_word)
                cntr += 1

    return bars

In [ ]:
def build_dataset(lyrics, rhyme_list):
    '''Generate inputs in the correct shape (2x2 tensors)'''
    '''lyrics can be the original text or the already trained data'''
    dataset = []
    line_list = []
    # For each line in lyrics, make a list composed by:
    # - the line itself
    # - the output of syllables() for that line
    # - the output of rhyme (most common word endingssyllables of the line (values range from) the line, the syllables
    # line_list becomes a list of the line from the lyrics, the syllables for that line (either 0 or 1 since
    # syllables uses integer division by maxsyllables (16)), and then rhyme returns the most common
    # word endings of possible rhyming words
    for line in lyrics:
        line_list = [line, syllables(line), rhyme(line, rhyme_list)]
        dataset.append(line_list)

    x_data = []
    y_data = []

    for i in range(len(dataset) - 3):
        line1 = dataset[i][1:]
        line2 = dataset[i + 1][1:]
        line3 = dataset[i + 2][1:]
        line4 = dataset[i + 3][1:]

        # Add datapoints. Start from a list of two pairs of (syllable, rhyme_index)
        x = [line1[0], line1[1], line2[0], line2[1]]
        x = np.array(x)
        # Reshape as a 2x2 tensor to adhere with LSTM model requirements
        # Each row is now a [syllable, rhyme_index] pair
        x = x.reshape(2, 2)
        

        # Do the same for the target data
        y = [line3[0], line3[1], line4[0], line4[1]]
        y = np.array(y)
        y = y.reshape(2, 2)

        # Sanity check for the types
        if type(x) is np.ndarray and type(y) is np.ndarray:
            x_data.append(x)
            y_data.append(y)
        else:
            continue

    # Convert to array
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    
    return x_data, y_data

In [ ]:
def compose_rap(lines, rhyme_list, lyrics_file, model):
    rap_vectors = []

    # Check if any empty lines are present and remove them
    while "" in lyrics_file:
        lyrics_file.remove("")

    # Start generation process from a random line from lyrics_file.
    # Create a list of 2 lines
    initial_index = random.choice(range(len(lyrics_file) - 1))
    initial_lines = lyrics_file[initial_index:initial_index + 8]

    starting_input = []
    # For each line in initial_lines,
    # append a (syllable,rhyme_index) pair to starting_input
    for line in initial_lines:
        starting_input.append([syllables(line), rhyme(line, rhyme_list)])

    # Generate output predictions for starting_input
    # Perform sanity check of the type of starting_input to be sure it's float32
    starting_vectors = model.predict(np.array([starting_input]).flatten().reshape(4, 2, 2).astype('float32'))
    rap_vectors.append(starting_vectors)

    # Repeat the process for the desired length of the rap 
    for i in range(rap_length):
        rap_vectors.append(model.predict(np.array([rap_vectors[-1]]).flatten().reshape(4, 2, 2)))

    return rap_vectors

In [ ]:
'''def vectors_into_song(vectors, generated_lyrics, rhyme_list):
	def last_word_compare(rap, line2):
		penalty = 0 
		for line1 in rap:
			word1 = line1.split(" ")[-1]
			word2 = line2.split(" ")[-1]
			if word1[-1] in "?!,.":
				word1 = word1[:-1]
			if word2[-1] in "?!,.":
				word2 = word2[:-1]
			if word1 == word2:
				penalty += 0.2
		return penalty
	def calculate_score(vector_half, syllables, rhyme, penalty):
		desired_syllables = vector_half[0]
		desired_rhyme = vector_half[1]
		desired_syllables = desired_syllables * maxsyllables
		desired_rhyme = desired_rhyme * len(rhyme_list)
		score = 1.0 - abs(float(desired_syllables) - float(syllables)) + abs(float(desired_rhyme) - float(rhyme)) - penalty
		return score
	dataset = []
	for line in generated_lyrics:
		line_list = [line, syllables(line), rhyme(line, rhyme_list)]
		dataset.append(line_list)
	rap = []
	vector_halves = []
	for vector in vectors:
		vector_halves.append(list(vector[0][0])) 
		vector_halves.append(list(vector[0][1]))
	for vector in vector_halves:
		scorelist = []
		for item in dataset:
			line = item[0]
			if len(rap) != 0:
				penalty = last_word_compare(rap, line)
			else:
				penalty = 0
			total_score = calculate_score(vector, item[1], item[2], penalty)
			score_entry = [line, total_score]
			scorelist.append(score_entry)
		fixed_score_list = []
		for score in scorelist:
			fixed_score_list.append(float(score[1]))
		max_score = max(fixed_score_list)
		for item in scorelist:
			if item[1] == max_score:
				rap.append(item[0])
				for i in dataset:
					if item[0] == i[0]:
						dataset.remove(i)
						break
				break     
	return rap'''

'def vectors_into_song(vectors, generated_lyrics, rhyme_list):\n\tdef last_word_compare(rap, line2):\n\t\tpenalty = 0 \n\t\tfor line1 in rap:\n\t\t\tword1 = line1.split(" ")[-1]\n\t\t\tword2 = line2.split(" ")[-1]\n\t\t\tif word1[-1] in "?!,.":\n\t\t\t\tword1 = word1[:-1]\n\t\t\tif word2[-1] in "?!,.":\n\t\t\t\tword2 = word2[:-1]\n\t\t\tif word1 == word2:\n\t\t\t\tpenalty += 0.2\n\t\treturn penalty\n\tdef calculate_score(vector_half, syllables, rhyme, penalty):\n\t\tdesired_syllables = vector_half[0]\n\t\tdesired_rhyme = vector_half[1]\n\t\tdesired_syllables = desired_syllables * maxsyllables\n\t\tdesired_rhyme = desired_rhyme * len(rhyme_list)\n\t\tscore = 1.0 - abs(float(desired_syllables) - float(syllables)) + abs(float(desired_rhyme) - float(rhyme)) - penalty\n\t\treturn score\n\tdataset = []\n\tfor line in generated_lyrics:\n\t\tline_list = [line, syllables(line), rhyme(line, rhyme_list)]\n\t\tdataset.append(line_list)\n\trap = []\n\tvector_halves = []\n\tfor vector in vectors:\n\

In [ ]:
def vectors_into_song(vectors, generated_lyrics, rhyme_list):
    ###Convert vectors into lyrics.
    ###   vectors: 2x2 predicted bars generated by compose_rap()
    
    # Comparing the last words between the original and the generated lyrics
    # granting a penalty if they are the same.
    def last_word_compare(rap, line2):
        penalty = 0
        for line1 in rap:
            word1 = line1.split(" ")[-1]
            word2 = line2.split(" ")[-1]

            # remove punctuations
            if word1[-1] in "?!,.":
                word1 = word1[:-1]

            if word2[-1] in "?!,.":
                word2 = word2[:-1]

            if word1 == word2:
                penalty += 0.2

        return penalty

    # vector_half is a single [syllable, rhyme_index] pair
    # returns a score rating for a given line
    def calculate_score(vector_half, syllables, rhyme, penalty):
        desired_syllables = vector_half[0]
        desired_rhyme = vector_half[1]
        # number of syllables
        desired_syllables = desired_syllables * maxsyllables
        # index of rhymes
        desired_rhyme = desired_rhyme * len(rhyme_list)

        # score from 1 substracts the sum of the difference between
        # predicted syllables and generated syllables and the difference between
        # the predicted rhyme and generated rhyme and then subtracts the penalty
        score = 1.0 - (abs((float(desired_syllables) - float(syllables))) + abs(
            (float(desired_rhyme) - float(rhyme)))) - penalty

        return score

    # First generate a list of all the lines. Each of them is combined with its syllables
    # and its rhyme float value
    dataset = []
    for line in generated_lyrics:
        line_list = [line, syllables(line), rhyme(line, rhyme_list)]
        dataset.append(line_list)

    rap = []

    vector_halves = []
    # Then separate each vector into half (or one bar)
    # Each bar is composed by a (syllables, rhyme_index) pair
    for vector in vectors:
        vector_halves.append(list(vector[0][0]))
        vector_halves.append(list(vector[0][1]))

    # Score each vector against every generated line/bar (item) to find the most
    # suitable bar predicted (by looking at the highest score).
    # Add bar to final lyrics and remove from the dataset to avoid duplicates
    for vector in vector_halves:
        scorelist = []
        # For each genereated bar from Markov model in the dataset
        for item in dataset:

            line = item[0]

            #Start with penaly 0, then compare the last word to assign penalty
            if len(rap) != 0:
                penalty = last_word_compare(rap, line)
            else:
                penalty = 0

            # calculate line score
            total_score = calculate_score(vector, item[1], item[2], penalty)
            # Bind the total score of the line with itself and put it as an entry
            score_entry = [line, total_score]
            # Add the entry to a score list
            scorelist.append(score_entry)

        fixed_score_list = []
        # Take only the score from each entry and append to fixed_score_list
        for score in scorelist:
            fixed_score_list.append((score[1]))
        # Continue if fixed_score_list is empty 
        if not fixed_score_list:
            continue
        # # Retrieve line associated with the max score from fixed_score_list
        max_score = max(fixed_score_list)
        print(f'score: {max_score}')
        for item in scorelist:
            if item[1] == max_score:
                # Add bar to final lyrics
                rap.append(item[0])
                # Remove the line from the dataset to avoid duplication
                for i in dataset:
                    if item[0] == i[0]:
                        dataset.remove(i)
                        break
                break
    return rap


In [17]:
def main(depth):
    '''End-to-end function. Create the model, the rhyme list, the dataset.
       Train the model, generate bars, convert to vectors and then into lyrics.
       Print the generated lyrics.'''
    train_mode = True

    # Create the model
    model = make_model(depth)
    
    # Make a "copy" of the edited rap as a sanity check
    text_file = ed_rap
    
    # Check if any empty lines are present and remove them
    while "" in text_file:
        text_file.remove("")
    
    # Make a "copy" of text_file as a sanity check
    bars = text_file

    # Delete the first rhyme (empty rhyme '')
    rhyme_list = rhymeindex(bars)
    del rhyme_list[0]
    # For loop to test which rhymes are biased (chosen by the model but not human-generated lyrics)
    remove_list = ['y', 'ay', 'by', 'cy', 'dy', 'ey', 'fy', 'hy', 'ky', 'ly', 'my', 'ny', 'oy', 'py', 'ry', 'sy', 'uy', 'vy', 'wy', 'xy', 'yy', 'zy', 'z', 'az', 'dz', 'ez', 'gz', 'hz', 'iz', 'lz', 'mz', 'nz']
    for rhyme in remove_list:
      rhyme_list.remove(rhyme)
    
    # Print rhyme list
    print(f'rhyme_list: {rhyme_list}')

    # Build the dataset and convert to float32 array
    x_data, y_data = build_dataset(bars, rhyme_list)
    x_data = np.asarray(x_data).astype('float32')
    y_data = np.asarray(y_data).astype('float32')
    
    # Train the model
    model.fit(np.array(x_data), np.array(y_data),
          batch_size=32,
          epochs=epochs_to_train,
          verbose=1)
    
    # Generate lyrics
    bars = generate_lyrics(text_file)

    # Compose rap
    vectors = compose_rap(bars, rhyme_list, text_file, model)

    # Turn vectors into song
    rap = vectors_into_song(vectors, bars, rhyme_list)
    
    # Print generated lyrics
    for bar in rap:
        print(bar)

In [18]:
# Main script:
# 1. Define the network parameters:
# - depth: depth of the network
# - maxsyllables: maximum number of syllables for each line
# - rap_length: number of lines in the lyrics created
# - epochs: how many times the network is trained
# 2. Call main(depth) function
depth = 4
maxsyllables = 16
rap_length = 8
epochs_to_train = 2

main(depth)

Loading rhymes from list.rhymes
rhyme_list: ['0', '00', '10', '20', '30', '40', '50', '60', '80', '90', 'e0', '1', '01', '11', '21', '71', '91', 'a1', 'e1', 's1', '2', '02', '12', '22', '32', '42', '62', '72', '82', '92', 'k2', 'n2', 't2', 'u2', 'v2', 'w2', 'x2', '3', '03', '13', '23', '63', '73', '83', '93', 'c3', 'm3', 'o3', 'p3', 'x3', '4', '04', '14', '24', '34', '44', '54', '64', '74', '84', '94', 'b4', 'c4', 'x4', '5', '05', '15', '25', '35', '45', '55', '65', '75', '85', '95', 'c5', 'g5', 'x5', '6', '06', '16', '26', '36', '46', '56', '76', '86', '96', 'x6', '7', '07', '17', '47', '57', '67', '77', '87', '97', 'x7', '8', '08', '18', '28', '38', '48', '68', '88', '98', 'v8', 'x8', '9', '09', '19', '39', '59', '69', '79', '89', '99', 'b9', 'c9', 'h9', 'a', 'aa', 'ba', 'ca', 'da', 'ea', 'fa', 'ga', 'ha', 'ia', 'ja', 'ka', 'la', 'ma', 'na', 'oa', 'pa', 'ra', 'sa', 'ta', 'ua', 'va', 'wa', 'ya', 'za', 'ça', 'b', 'ab', 'cb', 'db', 'eb', 'ib', 'lb', 'mb', 'nb', 'ob', 'qb', 'rb', 'tb', '